In [1]:
"""Introduction to pytorch computation graph"""
from functools import wraps
from decorator import decorator

import nographs as nog
from graphviz import Digraph

import torch
# import torch.nn as nn
# import torch.nn.functional as F
from torchvision import models

In [2]:
nn_model = models.convnext_tiny()
#nn_model

In [3]:
node_style ={"style": 'filled',            "shape": 'box',
             "align": 'left',              "fontsize": '12',
             "ranksep":'0.01',             "height":'0.02',   "width":'0.04',}
gr_style = {"linelength": '16', "rankdir": 'LR'}#"landscape":'True',

def adapter_graphviz( wrapped ):
    """translation to GraphViz visitor"""
    def node_id( item ):
        return str(hash(item))
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @wraps( wrapped )
    def wrapper_bg( _item, _trav ):
        dot.node( node_id(_item), str(_item)[:20] )
        for result in wrapped(_item, _trav):
            dot.edge( node_id(_item), node_id(result[0]), str(result[1])[:20], )
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg

#### The translation stage. For each node performing list of transformation, repeatedly, while we can

In [11]:
# def process_stream( nodes, process,  ):
#     for node, is_already_changed in nodes:
#         results, hot = process( node )
#         if hot:
#             yield from ( (r,True) for r in results )
#         else:
#             yield node, is_already_changed

stop_words = {'__dir__', '__str__','__repr__','__doc__','__hash__'}
stop_words = stop_words | {'__delattr__', '__del__',}

def filter_boring( node ):
    """Decorator for filtering, lets shrink graph a bit"""
    print(f"111111111111111{node=}")
    if node not in stop_words:
        yield node, False

def expand_iterables( item ):
    """Decorator for sequenses substitution"""
    if hasattr( item, '__iter__' ):
        yield from ( (i,True) for i in item )
    else:
        yield node, False

def expand_call( item ):
    """Decorator for following call"""
    if hasattr( item, '__call__' ):
        yield item(), True
    else:
        yield item, False

# def do_stage( nodes, stage_transforms ):
#     print(f"33333")
#     # annotate node, was it changed (and need to recheck)
#     nodes = ( (n,False) for n in nodes )
#     repeat_nodes = []
#     for node, changed in nodes:
#         if changed:
#             repeat_nodes.append( (node,False) )
#         else:
#             yield node
    #nodes = repeat_nodes

def do_stage( nodes, stage_transforms ):
    print(f"33333")
    # annotate node, was it changed (and need to recheck)
    nodes = ( (n,False) for n in nodes )
    # while nodes needed to process
    while len(list(nodes)) >0:
        # perform sequence once
        for transform in stage_transforms:
            print(f"22222222{nodes=}")
            nodes = transform( nodes )
            print(f"66666666{nodes=}")
        # sort out not changed and repeat
        repeat_nodes = []
        for node, changed in nodes:
            if changed:
                repeat_nodes.append( (node,False) )
            else:
                yield node
        nodes = repeat_nodes

# def filter_boring( stream ):
#     """Decorator for filtering, lets shrink graph a bit"""
#     stop_words = {'__dir__', '__str__','__repr__','__doc__','__hash__'}
#     yield from filter( lambda e:e[1] not in stop_words, stream )

In [12]:
def trace_show(x):
    print( "+++++++++" )
    return x

@decorator
def stage_1( wrapped, item, _trav ):
    print(f"444444 item")
    yield from do_stage( wrapped( trace_show(item), _trav ), 
                        [filter_boring, expand_iterables, expand_call] )

#     """Decorator for sequenses substitution"""
#     # print(f"{hasattr( item, '__iter__' )} {item=}")
#     if hasattr( item, '__iter__' ):
#         for sub in item:
#             yield from expand_iterables( wrapped, sub, _trav )
#     else:
#         yield from wrapped( item, _trav )

In [13]:
@adapter_graphviz
@stage_1
def all_attr(var, _):
    for attr_name, _ in zip(dir(var), range(3)):
        print(attr_name)
        attr = getattr(var, attr_name)
        if attr is not None: #hasattr( attr, "__hash__"):
            #print((str(attr), str(attr_name)), type(attr))
            yield (str(attr), str(attr_name))
        try:
            pass
        except:
            print("$$$trouble")
            continue

trav_forward = nog.TraversalBreadthFirst(next_labeled_edges=all_attr)
trav_forward.start_from( start_vertices=list(nn_model.children()), build_paths=True )
#trav_forward.start_from( nn_model.children(), build_paths=True )
for _, _ in zip(trav_forward, range(10)):
    pass

444444 item
+++++++++
33333
T_destination
__annotations__
__call__
22222222nodes=<generator object do_stage.<locals>.<genexpr> at 0x0000013F5E1964A0>
66666666nodes=<generator object filter_boring at 0x0000013F5E196F20>
22222222nodes=<generator object filter_boring at 0x0000013F5E196F20>
66666666nodes=<generator object expand_iterables at 0x0000013F5E196660>
22222222nodes=<generator object expand_iterables at 0x0000013F5E196660>
66666666nodes=<generator object expand_call at 0x0000013F5E196E40>


TypeError: 'generator' object is not subscriptable

In [ ]:
print(all_attr.dot.source)              # pylint: disable=pointless-statement
all_attr.dot

In [ ]:
dir(nn_model)
type(nn_model)
type(nn_model).__bases__

In [ ]:
#list(nn_model.children())

In [ ]:
issubclass(type(nn_model), torch.nn.modules.module.Module)

In [ ]:
#dir(nn_model)